# ホームケージ訓練用コマンド集

## touch_rect.py
矩形をタッチすると報酬がもらえる

In [ ]:
!python touch_rect_v3.py --serial-port COM4 \
    --kiosk --touch-only --fullscreen --auto-fullscreen-rect \
    --iti-min-ms 1000 --iti-max-ms 5000 --rect-mode auto \
    --info --min-side-px 150 --wait-release-timeout-ms 2000

## touch_rect_step2.py
矩形の範囲外を数回タッチするとエラーになる

In [14]:
!python touch_rect_step2.py --serial-port COM4 \
    --touch-only --rect-mode square_custom \
    --square-px 200 --min-side-px 1 --fullscreen \
    --iti-min-ms 1000 --iti-max-ms 5000 \
    --info --min-side-px 150 --wait-release-timeout-ms 2000\
    --max-outside-before-fail 5


pygame 2.6.1 (SDL 2.32.54, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
[INFO] Saved CSV: logs\touch_rect_log_20250924_155004.csv


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


## touch_rect_random.py

In [ ]:
!python touch_rect_random.py --serial-port COM4 --kiosk --touch-only --square-px 100 --iti-min-ms 500 --iti-max-ms 2000 --margin-px 50

### 事前にwindowsのエッジスワイプを無効にしておくこと

グループポリシーエディタを開いて「コンピュータの構成」「管理用テンプレート」「windowsコンポーネント」「画面の端のUI」

了解！`touch_rect.py` の主なコマンドライン引数を用途別にまとめます。角括弧内は既定値です。

# 基本

* `--serial-port <PORT>`（必須）
  TTL を送る Arduino 等のシリアルポート。例: `COM4`, `/dev/ttyACM0`
* `--serial-baud <INT>` \[115200]
  シリアル通信のボーレート。

# 画面・UI

* `--fullscreen`
  フルスクリーン表示。
* `--window-w <INT>` \[1280], `--window-h <INT>` \[720]
  ウィンドウ表示時のサイズ（`--fullscreen` なしのとき有効）。
* `--kiosk`
  キオスクモード。**自動で `--fullscreen`、マウス非表示、入力グラブ**を有効化。
* `--info`
  画面左上に状態・矩形サイズ・成功数などを表示。
* `--show-box`
  刺激矩形の外枠（薄灰色）を描画。

# 入力

* `--touch-only`
  マウス系イベントを**ブロック**し、タッチ（`FINGER*`）入力のみに限定。

# 刺激（矩形）サイズ・モード

* `--rect-mode {auto|square|square_custom}` \[auto]
  矩形生成モード（いずれも画面中央に配置）。

  * `auto` / `square`：短辺×`size_frac` の正方形。
  * `square_custom`：`--square-px` で指定したピクセルの正方形。
* `--square-px <INT>` \[なし]
  `square_custom` の一辺ピクセル。
* `--initial-size-frac <FLOAT>` \[1.0]
  初期の辺長（画面短辺に対する割合）。例: 0.5 → 短辺の 50%。
* `--min-size-frac <FLOAT>` \[0.10]
  縮小してもこれ以下の**割合**にはしない下限。
* `--min-side-px <INT>` \[なし]
  縮小してもこれ以下の**ピクセル**にはしない下限。
  ※`min-size-frac` と併用時は「より大きい方」を採用（見失い防止）。
* `--shrink-every <INT>` \[10]
  この成功試行数ごとにサイズを縮小（段階的に難易度アップ）。
* `--shrink-factor <FLOAT>` \[0.8]
  縮小率。例: 0.8 なら 20% 縮小。
* `--auto-fullscreen-rect`
  `--fullscreen` かつ辺長が画面短辺以上のとき**矩形を全面**にする。
  ※デフォルトでは**全面化しません**（情報UIが隠れるのを防止）。

# 色

* `--rect-rgb R G B` \[0 160 255]
  刺激矩形の色（0–255）。
* `--bg-rgb R G B` \[0 0 0]
  背景色。

# ITI（試行間インターバル）・サウンド

* `--iti-min-ms <INT>` \[1000], `--iti-max-ms <INT>` \[1000]
  ITI をミリ秒でランダム設定（`min`〜`max` の一様乱数）。
  例: `--iti-min-ms 500 --iti-max-ms 2000`
* `--beep-freq <INT>` \[1000], `--beep-ms <INT>` \[100], `--beep-volume <FLOAT>` \[0.6]
  反応時のビープ音（周波数Hz、長さms、音量0–1）。

# フェイルセーフ（取りこぼし対策）

* `--wait-release-timeout-ms <INT>` \[2000]
  反応後の **「離し待ち（WAIT\_RELEASE）」がこの時間を超えたら**、
  指が離れた扱いにして**強制的に次試行へ**（まれな `FINGERUP` 取りこぼし対策）。

# ログ

* `--out-dir <PATH>` \[`logs`]
  CSV 出力先ディレクトリ。`touch_rect_log_YYYYMMDD_HHMMSS.csv` を保存。

---

## よくある組み合わせ例

### ご指定の条件＋見失い防止＋詰まり対策

```bash
python touch_rect.py \
  --serial-port COM4 \
  --kiosk --touch-only --fullscreen \
  --iti-min-ms 500 --iti-max-ms 2000 \
  --rect-mode auto --info \
  --min-side-px 140 \
  --wait-release-timeout-ms 2000
```

### きめ細かく段階難易度

```bash
--initial-size-frac 0.8 --min-size-frac 0.2 --shrink-every 5 --shrink-factor 0.9
```

### カスタム固定サイズ（中央に正方形 300px）

```bash
--rect-mode square_custom --square-px 300
```

---

## 補足仕様（動作のコツ）

* `--kiosk` は **`--fullscreen` を自動有効化**、マウス非表示、入力グラブを行います。
* `--touch-only` はマウス入力を完全ブロックするので、デバッグ中にマウスクリックで試す場合は外してください。
* 矩形の全面化は **`--auto-fullscreen-rect` を付けたときだけ**行われます。UI を隠したくない場合は付けないでください。
* 縮小の下限は **`min-side-px` と `min-size-frac` のうち大きい方**が適用されます（見えなくなる事故を防ぐため）。

他にも運用に合わせて既定値を調整します。必要なら「初期サイズを徐々に戻す」「一定試行ごとにサイズを一段戻す」などのバリアフリー調整も入れられます。
